In [1]:
import music21

In [2]:
from music21 import converter

score_fn = '../music_score/chihwapyeong_evom.musicxml'

score = converter.parse(score_fn)

In [9]:
from typing import List
class Gnote:
    def __init__(self, note: music21.note.Note, part_idx:int) -> None:
        self.note = [note]  

        self.pitch = note.pitch.ps
        self.tie = note.tie
        self.part_idx = part_idx
        if part_idx == 0:
            self.pitch -= (9+12) # C4 -> EB2
        self.measure_number = note.measureNumber
        self.offset = note.offset
        self.measure_offset = 0.0 # dummy value, float로 함!(offset: float)
        # self.duration = sum([note.duration.quarterLength for note in self.note])
        # self.dynamic = 0.0
        
    def __add__(self, note: music21.note.Note):
        # assert self.pitch == note.pitch.ps
        self.note.append(note)
        return self

    @property
    def duration(self):
        return sum([note.duration.quarterLength for note in self.note])

    @property
    def end_measure_number(self):
        return self.note[-1].measureNumber
  
    
    def __repr__(self) -> str:
        return f'Gnote: {self.pitch}, {self.duration}, {self.measure_offset}'


def apply_tie(notes: List[music21.note.Note], part_idx) -> List[music21.note.Note]:
    tied_notes = []
    for note in notes:
        if note.tie is not None:
            if note.tie.type == 'start':
                tied_notes.append(Gnote(note, part_idx))
            elif note.tie.type == 'continue':
                tied_notes[-1] +=  note
            elif note.tie.type == 'stop':
                tied_notes[-1] += note
            else:
                raise ValueError(f'Unknown tie type: {note.tie.type}')
        else:
            tied_notes.append(Gnote(note, part_idx))
        
    return tied_notes


In [11]:
for idx, part in enumerate(score.parts):
  tied_notes = apply_tie(part.flat.notes, idx)
  break

In [12]:
tied_notes[:10]

[Gnote: 54.0, 0.25, 0.0,
 Gnote: 54.0, 1.25, 0.0,
 Gnote: 54.0, 1.0, 0.0,
 Gnote: 56.0, 0.5, 0.0,
 Gnote: 61.0, 0.25, 0.0,
 Gnote: 56.0, 0.75, 0.0,
 Gnote: 56.0, 0.75, 0.0,
 Gnote: 47.0, 0.25, 0.0,
 Gnote: 54.0, 1.25, 0.0,
 Gnote: 56.0, 0.25, 0.0]

In [13]:
stream = music21.stream.Stream()
for note in tied_notes:
    stream.append(music21.note.Note(note.pitch, quarterLength=note.duration))

In [14]:
stream.write('musicxml', 'tie_test.musicxml')

PosixPath('/home/teo/userdata/SejongMusic/notebook/tie_test.musicxml')